In [40]:
import sys
sys.path.append("..")
import tensorflow as tf
import keras
import numpy as np
import keras_utils
%matplotlib inline
import matplotlib.pyplot as plt
import keras_tqdm

In [41]:
from keras.datasets import cifar10
(x_train,y_train) ,(x_test,y_test) = cifar10.load_data()

In [42]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [43]:
NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

In [44]:
x_train_norm = (x_train/255) - 0.5
x_test_norm = (y_test/255) - 0.5
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train,num_classes=NUM_CLASSES)
y_test_encoded = to_categorical(y_test,num_classes=NUM_CLASSES)

In [45]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation,Dropout
from keras.layers.advanced_activations import LeakyReLU

In [46]:
def make_model():
    
    model = Sequential()
    
    model.add(Conv2D(16,(3,3),strides=(1,1),input_shape=(32,32,3),padding="same"))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(32,(3,3),strides=(1,1),padding="same"))
    model.add(LeakyReLU(0.1))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25,noise_shape=None,seed=0))
    
    model.add(Conv2D(32,(3,3),strides=(1,1),padding="same"))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(64,(3,3),strides=(1,1),padding="same"))
    model.add(LeakyReLU(0.1))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25,noise_shape=None,seed=0))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5,noise_shape=None,seed=0))
    model.add(Dense(NUM_CLASSES))
    
    model.add(Activation("softmax"))
    
    return model
    
    
    
    

In [47]:

model = make_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 32, 32, 16)        448       
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 16, 16, 32)       

In [48]:
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 10


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 10-way classification
    optimizer=keras.optimizers.Adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [49]:
model_filename='cifar.{0:03d}.hdf5'
last_finished_epoch = None

In [54]:
# fit model
model.fit(x_train_norm,y_train_encoded,epochs=EPOCHS)


Epoch 1/10
1563/1563 [==============================] - 228s 146ms/step - loss: 1.3504 - accuracy: 0.5149
Epoch 2/10
1563/1563 [==============================] - 227s 145ms/step - loss: 0.9415 - accuracy: 0.6716
Epoch 3/10
1563/1563 [==============================] - 228s 146ms/step - loss: 0.8266 - accuracy: 0.7120
Epoch 4/10
1563/1563 [==============================] - 222s 142ms/step - loss: 0.7431 - accuracy: 0.7397
Epoch 5/10
1563/1563 [==============================] - 228s 146ms/step - loss: 0.6993 - accuracy: 0.7561
Epoch 6/10
1563/1563 [==============================] - 229s 147ms/step - loss: 0.6647 - accuracy: 0.7676
Epoch 7/10
1563/1563 [==============================] - 228s 146ms/step - loss: 0.6261 - accuracy: 0.7813
Epoch 8/10
1563/1563 [==============================] - 229s 146ms/step - loss: 0.6042 - accuracy: 0.7878
Epoch 9/10
1563/1563 [==============================] - 241s 154ms/step - loss: 0.5781 - accuracy: 0.8002
Epoch 10/10
1563/1563 [=======================

In [57]:
model.save_weights("weights.h5")


In [58]:
model.load_weights("weights.h5")

In [62]:
# make test predictions
y_pred_test = model.predict(x_test_norm)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
y_pred_test_max_probas = np.max(y_pred_test, axis=1)

ValueError: in user code:

    C:\Users\devro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\devro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\devro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\devro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\devro\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1122 predict_step  **
        return self(x, training=False)
    C:\Users\devro\AppData\Roaming\Python\